# 2022 Flatiron Machine Learning x Science Summer School

## Step 4: Train MLP with $L_1$ regularization on latent features

### Step 4.1: Check $L_1$ regularization parameter

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

import torch
import wandb

from srnet import SRNet, SRData
import srnet_utils as ut

In [ ]:
# set wandb project
wandb_project = "41-l1-study-F00"

In [ ]:
# define hyperparameters
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": "MLP",
#         "lat_size": 16,
#         },
#     "epochs": 10000,
#     "runtime": None,
#     "batch_size": 64,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "shuffle": True,
# }

In [ ]:
# define hyperparameter study
hp_study = {
    "method": "grid", # random, bayesian
    #"metric": {
    #    "name": "val_loss",
    #    "goal": "minimize",
    #},
    "parameters": {
        "l1": {
            "values": [1e-6, 1e-4, 1e-2, 1e-1]
        }
    }
}

In [ ]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

In [ ]:
# download data from wandb
file_ext = ".pkl"

api = wandb.Api()

runs = api.runs(wandb_project)
for run in runs:
    for f in run.files():
        if f.name[-len(file_ext):] == file_ext and not os.path.isfile(f.name):
            print(f"Downloading {os.path.basename(f.name)}.")
            run.file(f.name).download()

In [ ]:
# plot losses
save_names = ["F00_l1", "F00_conv1k"]
save_path = "models"
models = ut.plot_losses(save_names, save_path="models", excl_names=["gc"])

In [ ]:
# print losses
states = {}
model_ext = ".pkl"
for model_name in models:
    state = joblib.load(os.path.join(save_path, model_name + model_ext))
    print(f"{model_name.split('.')[0].split('_')[-1]}:\t{state['total_train_loss']:.3e} {state['total_val_loss']:.3e}")

Notes:

* A $L_1$ regularization parameter of `1e-3` seems to be an optimum for the validation error

In [ ]:
# load data
data_path = "data_1k"

in_var = "X00"
lat_var = "G00"
target_var = "F00"

mask_ext = ".mask"
masks = joblib.load(os.path.join(data_path, in_var + mask_ext))     # TODO: create mask if file does not exist

train_data = SRData(data_path, in_var, lat_var, target_var, masks["train"])
val_data = SRData(data_path, in_var, lat_var, target_var, masks["val"])

In [ ]:
# latent feature variance overview
model_path = "models"
model_ext = ".pkl"

models = [
    "srnet_model_F00_conv1k",
    "srnet_model_F00_l1_1e-06",
    "srnet_model_F00_l1_1e-04",
    "srnet_model_F00_l1_1e-03",
    "srnet_model_F00_l1_1e-02",
    "srnet_model_F00_l1_1e-01",
]

for model_name in models:
    print(model_name)
    model = ut.load_model(model_name + model_ext, model_path, SRNet)
    
    with torch.no_grad():
        preds, acts = model(train_data.in_data, get_lat=True)
        
    all_nodes = ut.get_node_order(acts, show=True)
    print("")

Notes:

* A $L_1$ regularization parameter of `1e-3` yields three high variance latent features

In [ ]:
model_name = "srnet_model_F00_l1_1e-03"

model = ut.load_model(model_name + model_ext, model_path, SRNet)

with torch.no_grad():
    preds, acts = model(train_data.in_data, get_lat=True)
    
all_nodes = ut.get_node_order(acts, show=True)

In [ ]:
nodes = all_nodes[:3]

In [ ]:
# select plotting data
x_data = train_data.in_data[:,0]
y_data = train_data.in_data[:,1]
z_data = [
    ("target", train_data.target_data),
    #("x**2", x_data**2), 
    #("cos(y)", np.cos(y_data)), 
    #("x*y", x_data * y_data),
]
plot_size = train_data.target_data.shape[0]

In [ ]:
ut.plot_acts(x_data, y_data, z_data, acts=acts, nodes=nodes, model=model, agg=False, plot_size=plot_size)

In [ ]:
corr_data = [
    ("x**2", x_data**2), 
    ("cos(y)", np.cos(y_data)), 
    ("x*y", x_data * y_data),
]

In [ ]:
ut.node_correlations(acts, nodes, corr_data, nonzero=True)

Notes:

* The high variance latent features do not split into the desired latent functions

### Step 4.1: Train MLP with $L_1$ regularization for `F06`

In [ ]:
# set wandb project
wandb_project = "42-l1-study-F06"

In [ ]:
# define hyperparameters
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": "MLP",
#         "lat_size": 16,
#         },
#     "epochs": 10000,
#     "runtime": None,
#     "batch_size": 64,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "a1": 0.0,
#     "a2": 0.0,
#     "gc": 0.0,
#     "shuffle": True,
# }

In [ ]:
# define hyperparameter study
hp_study = {
    "method": "grid", # random, bayesian
    #"metric": {
    #    "name": "val_loss",
    #    "goal": "minimize",
    #},
    "parameters": {
        "l1": {
            "values": [1e-4, 1e-3, 1e-2]
        }
    }
}

In [ ]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

In [ ]:
# download data from wandb
file_ext = ".pkl"

api = wandb.Api()

runs = api.runs(wandb_project)
for run in runs:
    for f in run.files():
        if f.name[-len(file_ext):] == file_ext and not os.path.isfile(f.name):
            print(f"Downloading {os.path.basename(f.name)}.")
            run.file(f.name).download()

In [ ]:
# plot losses
save_names = ["F06_l1", "F06_conv1k"]
save_path = "models"
models = ut.plot_losses(save_names, save_path="models", excl_names=["gc"])

In [ ]:
# print losses
model_ext = ".pkl"
states = {}
for model_name in models:
    state = joblib.load(os.path.join(save_path, model_name + model_ext))
    print(f"{model_name.split('.')[0].split('_')[-1]}:\t{state['total_train_loss']:.3e} {state['total_val_loss']:.3e}")

In [ ]:
# load data
data_path = "data_1k"

in_var = "X06"
lat_var = "G06"
target_var = "F06"

mask_ext = ".mask"
masks = joblib.load(os.path.join(data_path, in_var + mask_ext))

train_data = SRData(data_path, in_var, lat_var, target_var, masks["train"])
val_data = SRData(data_path, in_var, lat_var, target_var, masks["val"])

In [ ]:
# latent feature variance overview
model_path = "models"

for model_name in models:
    print(model_name)
    model = ut.load_model(model_name + model_ext, model_path, SRNet)
    
    with torch.no_grad():
        preds, acts = model(train_data.in_data, get_lat=True)
        
    all_nodes = ut.get_node_order(acts, show=True)
    print("")

Notes:

* Interestingly, while `1e-03` is clearly too little regularization, `5e-03` seems to be already too much

* `3e-03` seems to be alright, however, the sensitivity is surprising

In [ ]:
# load model
model_name = "srnet_model_F06_l1_3e-03.pkl"
model_path = "models"

model = ut.load_model(model_name, model_path, SRNet)

In [ ]:
# get predictions
with torch.no_grad():
    preds, acts = model(train_data.in_data, get_lat=True)

In [ ]:
# get latent feature variance
all_nodes = ut.get_node_order(acts, show=True)

In [ ]:
nodes = all_nodes[:4]

In [ ]:
# select data
x0_data = train_data.in_data[:,0]
x3_data = train_data.in_data[:,3]
x5_data = train_data.in_data[:,5]
x7_data = train_data.in_data[:,7]

corr_data = [
    ("x0**2", x0_data**2), 
    ("cos(x3)", np.cos(x3_data)), 
    ("x5*x7", x5_data * x7_data),
    ("x0", x0_data),
    ("x3", x3_data),
    ("x5", x5_data),
    ("x7", x7_data),
]

In [ ]:
ut.node_correlations(acts, nodes, corr_data, nonzero=True)

Notes:

* Despite being now separable, the high variance latent features do not split into the desired latent functions